# Climatrix 2 

Includes better data-preprocessing & Machine Learning Training

In [1]:
import os 
import shutil
from pathlib import Path 


import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as plotly 

In [2]:
import numpy as np
import pandas as pd

def preprocess_weather_csv_hourly(csv_path: str):
    # -----------------------------
    # Load + quality filtering
    # -----------------------------
    df = pd.read_csv(csv_path)

    df["time"] = pd.to_datetime(df["time"], errors="raise")
    df = df.sort_values("time").drop_duplicates(subset=["time"])

    # Remove low-quality rows: cattr < 7
    if "cattr" in df.columns:
        df = df[df["cattr"] >= 7].copy()

    # Keep only what you need (cidx removed implicitly)
    keep = ["time", "temp", "humi", "pres"]
    df = df[keep].copy()
    df = df.set_index("time")

    # -----------------------------
    # Resample to hourly
    # -----------------------------
    df_h = df.resample("1H").mean()

    # Fill missing
    df_h[["temp", "humi", "pres"]] = df_h[["temp", "humi", "pres"]].interpolate(method="time", limit=3)
    df_h = df_h.ffill(limit=2)

    # -----------------------------
    # Anomaly handling
    # -----------------------------
    bounds = {"temp": (10.0, 45.0), "humi": (0.0, 100.0), "pres": (930.0, 1070.0)}
    for col, (lo, hi) in bounds.items():
        bad = (df_h[col] < lo) | (df_h[col] > hi)
        df_h.loc[bad, col] = np.nan

    spike_thresh = {"temp": 5.0, "humi": 30.0, "pres": 10.0}  # per hour
    for col, thr in spike_thresh.items():
        jump = df_h[col].diff().abs()
        df_h.loc[jump > thr, col] = np.nan

    df_h[["temp", "humi", "pres"]] = df_h[["temp", "humi", "pres"]].interpolate(method="time", limit=3)
    df_h = df_h.ffill(limit=2).bfill(limit=2)

    # -----------------------------
    # Features
    # -----------------------------
    df_feat = df_h.copy()

    # time features
    hour = df_feat.index.hour
    df_feat["hour_sin"] = np.sin(2 * np.pi * hour / 24.0)
    df_feat["hour_cos"] = np.cos(2 * np.pi * hour / 24.0)
    df_feat["dow"] = df_feat.index.dayofweek

    # lags + rolling
    for c in ["temp", "humi", "pres"]:
        df_feat[f"{c}_lag1"] = df_feat[c].shift(1)
        df_feat[f"{c}_lag2"] = df_feat[c].shift(2)
        df_feat[f"{c}_lag3"] = df_feat[c].shift(3)
        df_feat[f"{c}_roll3_mean"] = df_feat[c].rolling(3).mean()
        df_feat[f"{c}_roll6_mean"] = df_feat[c].rolling(6).mean()

    # targets: next hour
    df_feat["y_temp"] = df_feat["temp"].shift(-1)
    df_feat["y_humi"] = df_feat["humi"].shift(-1)
    df_feat["y_pres"] = df_feat["pres"].shift(-1)

    df_feat = df_feat.dropna()

    X = df_feat.drop(columns=["y_temp", "y_humi", "y_pres"])
    y = df_feat[["y_temp", "y_humi", "y_pres"]]

    return df_h, X, y


def time_split(X, y, train=0.7, val=0.15):
    n = len(X)
    n_train = int(n * train)
    n_val = int(n * val)

    X_train, y_train = X.iloc[:n_train], y.iloc[:n_train]
    X_val, y_val = X.iloc[n_train:n_train+n_val], y.iloc[n_train:n_train+n_val]
    X_test, y_test = X.iloc[n_train+n_val:], y.iloc[n_train+n_val:]

    return X_train, y_train, X_val, y_val, X_test, y_test


In [3]:
csv_path = "/Users/saikeerthan/Coding/NYP/IOTA/IoT_Weather_project/model-training/datasets/official_data.csv"

df_hourly_clean, X, y = preprocess_weather_csv_hourly(csv_path)
X_train, y_train, X_val, y_val, X_test, y_test = time_split(X, y)

print("df_hourly_clean:", df_hourly_clean.shape)
print("X:", X.shape, "y:", y.shape)
print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

# See the dataframe
display(df_hourly_clean.head(10))
display(X.head(5))
display(y.head(5))


df_hourly_clean: (207, 3)
X: (201, 21) y: (201, 3)
Train: (140, 21) Val: (30, 21) Test: (31, 21)


/var/folders/_r/50s4djvj3g5_yz8zn0m59nbm0000gn/T/ipykernel_59704/816512574.py:25: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



,temp,humi,pres
time,,,
2026-01-24 20:00:00,28.612500,74.000000,1016.968659
2026-01-24 21:00:00,28.508333,75.000000,1017.537598
2026-01-24 22:00:00,28.881818,75.000000,1017.855003
2026-01-24 23:00:00,28.916667,75.083333,1017.733541
2026-01-25 00:00:00,28.975000,76.000000,1017.420471
2026-01-25 01:00:00,28.916667,76.000000,1017.089579
2026-01-25 02:00:00,28.883333,76.500000,1016.587077
2026-01-25 03:00:00,28.725000,76.833333,1016.238444
2026-01-25 04:00:00,28.650000,76.750000,1016.038534


,temp,humi,pres,hour_sin,hour_cos,dow,temp_lag1,temp_lag2,temp_lag3,temp_roll3_mean,...,humi_lag1,humi_lag2,humi_lag3,humi_roll3_mean,humi_roll6_mean,pres_lag1,pres_lag2,pres_lag3,pres_roll3_mean,pres_roll6_mean
time,,,,,,,,,,,,,,,,,,,,,
2026-01-25 01:00:00,28.916667,76.000000,1017.089579,0.258819,0.965926,6,28.975000,28.916667,28.881818,28.936111,...,76.000000,75.083333,75.000000,75.694444,75.180556,1017.420471,1017.733541,1017.855003,1017.414530,1017.434142
2026-01-25 02:00:00,28.883333,76.500000,1016.587077,0.500000,0.866025,6,28.916667,28.975000,28.916667,28.925000,...,76.000000,76.000000,75.083333,76.166667,75.597222,1017.089579,1017.420471,1017.733541,1017.032376,1017.370545
2026-01-25 03:00:00,28.725000,76.833333,1016.238444,0.707107,0.707107,6,28.883333,28.916667,28.975000,28.841667,...,76.500000,76.000000,76.000000,76.444444,75.902778,1016.587077,1017.089579,1017.420471,1016.638367,1017.154019
2026-01-25 04:00:00,28.650000,76.750000,1016.038534,0.866025,0.500000,6,28.725000,28.883333,28.916667,28.752778,...,76.833333,76.500000,76.000000,76.694444,76.194444,1016.238444,1016.587077,1017.089579,1016.288018,1016.851274
2026-01-25 05:00:00,28.509091,76.090909,1016.129350,0.965926,0.258819,6,28.650000,28.725000,28.883333,28.628030,...,76.750000,76.833333,76.500000,76.558081,76.362374,1016.038534,1016.238444,1016.587077,1016.135443,1016.583909


,y_temp,y_humi,y_pres
time,,,
2026-01-25 01:00:00,28.883333,76.500000,1016.587077
2026-01-25 02:00:00,28.725000,76.833333,1016.238444
2026-01-25 03:00:00,28.650000,76.750000,1016.038534
2026-01-25 04:00:00,28.509091,76.090909,1016.129350
2026-01-25 05:00:00,28.545455,75.000000,1016.581077


## Machine Learning Training

In [6]:
# If imports fail, uncomment the pip installs and run once.

# !pip install -q lightgbm
# !pip install -q xgboost
# !pip install -q catboost

from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor


In [7]:
def eval_model(name, model, X_tr, y_tr, X_va, y_va):
    model.fit(X_tr, y_tr)
    pred = model.predict(X_va)

    # overall
    mae_all = mean_absolute_error(y_va, pred)
    rmse_all = np.sqrt(mean_squared_error(y_va, pred))

    # per target
    cols = ["temp", "humi", "pres"]
    mae_each = {}
    rmse_each = {}
    for i, c in enumerate(cols):
        mae_each[c] = mean_absolute_error(y_va.iloc[:, i], pred[:, i])
        rmse_each[c] = np.sqrt(mean_squared_error(y_va.iloc[:, i], pred[:, i]))

    out = {
        "model": name,
        "MAE_mean": mae_all,
        "RMSE_mean": rmse_all,
        "MAE_temp": mae_each["temp"],
        "MAE_humi": mae_each["humi"],
        "MAE_pres": mae_each["pres"],
        "RMSE_temp": rmse_each["temp"],
        "RMSE_humi": rmse_each["humi"],
        "RMSE_pres": rmse_each["pres"],
    }
    return out, model


models = {
    "LightGBM": MultiOutputRegressor(
        LGBMRegressor(
            n_estimators=800,
            learning_rate=0.03,
            num_leaves=63,
            subsample=0.9,
            colsample_bytree=0.9,
            random_state=42,
        )
    ),
    "XGBoost": MultiOutputRegressor(
        XGBRegressor(
            n_estimators=900,
            learning_rate=0.03,
            max_depth=6,
            subsample=0.9,
            colsample_bytree=0.9,
            reg_lambda=1.0,
            objective="reg:squarederror",
            random_state=42,
            n_jobs=-1,
        )
    ),
    "CatBoost": MultiOutputRegressor(
        CatBoostRegressor(
            iterations=1200,
            learning_rate=0.03,
            depth=8,
            loss_function="RMSE",
            verbose=0,
            random_seed=42,
        )
    ),
}

results = []
fitted = {}

for name, m in models.items():
    r, fitted_model = eval_model(name, m, X_train, y_train, X_val, y_val)
    results.append(r)
    fitted[name] = fitted_model

results_df = pd.DataFrame(results).sort_values("RMSE_mean")
display(results_df)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000175 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 879
[LightGBM] [Info] Number of data points in the train set: 140, number of used features: 21
[LightGBM] [Info] Start training from score 35.952115
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

,model,MAE_mean,RMSE_mean,MAE_temp,MAE_humi,MAE_pres,RMSE_temp,RMSE_humi,RMSE_pres
1,XGBoost,0.454822,0.597423,0.350373,0.661064,0.353025,0.420165,0.842330,0.429745
2,CatBoost,0.532654,0.749816,0.618213,0.613815,0.365933,0.744752,0.971439,0.433960
0,LightGBM,0.544663,0.761280,0.501652,0.829218,0.303119,0.607389,1.101949,0.394246


In [8]:
best_name = results_df.iloc[0]["model"]
best_model = fitted[best_name]

pred_test = best_model.predict(X_test)

test_mae = mean_absolute_error(y_test, pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, pred_test))

print("Best model:", best_name)
print("TEST  MAE_mean :", test_mae)
print("TEST  RMSE_mean:", test_rmse)

# Per-target on test
cols = ["temp", "humi", "pres"]
for i, c in enumerate(cols):
    mae_c = mean_absolute_error(y_test.iloc[:, i], pred_test[:, i])
    rmse_c = np.sqrt(mean_squared_error(y_test.iloc[:, i], pred_test[:, i]))
    print(f"{c.upper():4s} | MAE={mae_c:.4f} | RMSE={rmse_c:.4f}")


Best model: XGBoost
TEST  MAE_mean : 0.513457715511322
TEST  RMSE_mean: 0.7481013345441068
TEMP | MAE=0.3949 | RMSE=0.4822
HUMI | MAE=0.8432 | RMSE=1.1270
PRES | MAE=0.3023 | RMSE=0.4199


In [11]:
# getting r2 score for XGB

from sklearn.metrics import r2_score
import numpy as np

# Overall (averaged across outputs)
r2_overall = r2_score(y_test, pred_test, multioutput="uniform_average")
print("R2 overall (uniform avg):", r2_overall)

# Per target
names = ["temp", "humi", "pres"]
for i, n in enumerate(names):
    r2_i = r2_score(y_test.iloc[:, i], pred_test[:, i])
    print(f"R2_{n}: {r2_i:.4f}")


R2 overall (uniform avg): 0.7454497218132019
R2_temp: 0.6267
R2_humi: 0.7350
R2_pres: 0.8746


### Ensemble Tree Models

In [13]:
# =========================
# 0) Imports + installs
# =========================
import numpy as np
import pandas as pd

from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# If needed, run once in notebook:
# !pip install -q xgboost lightgbm catboost

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


# =========================
# 1) Preprocessing pipeline
# =========================
def preprocess_weather_csv_hourly(csv_path: str):
    df = pd.read_csv(csv_path)

    # Parse time
    df["time"] = pd.to_datetime(df["time"], errors="raise")
    df = df.sort_values("time").drop_duplicates(subset=["time"])

    # Filter: remove rows where cattr < 7
    if "cattr" in df.columns:
        df = df[df["cattr"] >= 7].copy()

    # Keep only what you need
    keep = ["time", "temp", "humi", "pres"]
    df = df[keep].copy().set_index("time")

    # Resample to hourly
    df_h = df.resample("1H").mean()

    # Fill missing (light)
    df_h[["temp","humi","pres"]] = df_h[["temp","humi","pres"]].interpolate(method="time", limit=3)
    df_h = df_h.ffill(limit=2)

    # Anomaly cleaning: physical bounds
    bounds = {
        "temp": (10.0, 45.0),
        "humi": (0.0, 100.0),
        "pres": (930.0, 1070.0),
    }
    for col, (lo, hi) in bounds.items():
        bad = (df_h[col] < lo) | (df_h[col] > hi)
        df_h.loc[bad, col] = np.nan

    # Spike cleaning: rate-of-change (per hour)
    spike_thresh = {"temp": 5.0, "humi": 30.0, "pres": 10.0}
    for col, thr in spike_thresh.items():
        jump = df_h[col].diff().abs()
        df_h.loc[jump > thr, col] = np.nan

    # Repair after anomaly marking
    df_h[["temp","humi","pres"]] = df_h[["temp","humi","pres"]].interpolate(method="time", limit=3)
    df_h = df_h.ffill(limit=2).bfill(limit=2)

    # Feature engineering
    df_feat = df_h.copy()

    # time features
    hour = df_feat.index.hour
    df_feat["hour_sin"] = np.sin(2*np.pi*hour/24.0)
    df_feat["hour_cos"] = np.cos(2*np.pi*hour/24.0)
    df_feat["dow"] = df_feat.index.dayofweek

    # lag + rolling features
    for c in ["temp","humi","pres"]:
        df_feat[f"{c}_lag1"] = df_feat[c].shift(1)
        df_feat[f"{c}_lag2"] = df_feat[c].shift(2)
        df_feat[f"{c}_lag3"] = df_feat[c].shift(3)
        df_feat[f"{c}_roll3_mean"] = df_feat[c].rolling(3).mean()
        df_feat[f"{c}_roll6_mean"] = df_feat[c].rolling(6).mean()

    # Targets: next hour
    df_feat["y_temp"] = df_feat["temp"].shift(-1)
    df_feat["y_humi"] = df_feat["humi"].shift(-1)
    df_feat["y_pres"] = df_feat["pres"].shift(-1)

    df_feat = df_feat.dropna()

    X = df_feat.drop(columns=["y_temp","y_humi","y_pres"])
    y = df_feat[["y_temp","y_humi","y_pres"]]

    return df_h, X, y


def time_split(X, y, train=0.7, val=0.15):
    n = len(X)
    n_train = int(n * train)
    n_val = int(n * val)

    X_train, y_train = X.iloc[:n_train], y.iloc[:n_train]
    X_val, y_val     = X.iloc[n_train:n_train+n_val], y.iloc[n_train:n_train+n_val]
    X_test, y_test   = X.iloc[n_train+n_val:], y.iloc[n_train+n_val:]

    return X_train, y_train, X_val, y_val, X_test, y_test



In [14]:

# =========================
# 2) Metrics helper
# =========================
def compute_metrics(y_true, y_pred):
    out = {}
    out["MAE_mean"]  = mean_absolute_error(y_true, y_pred)
    out["RMSE_mean"] = np.sqrt(mean_squared_error(y_true, y_pred))
    out["R2_mean"]   = r2_score(y_true, y_pred, multioutput="uniform_average")

    names = ["temp","humi","pres"]
    for i, n in enumerate(names):
        out[f"MAE_{n}"]  = mean_absolute_error(y_true.iloc[:, i], y_pred[:, i])
        out[f"RMSE_{n}"] = np.sqrt(mean_squared_error(y_true.iloc[:, i], y_pred[:, i]))
        out[f"R2_{n}"]   = r2_score(y_true.iloc[:, i], y_pred[:, i])
    return out


# =========================
# 3) Train base models
# =========================
csv_path = "/Users/saikeerthan/Coding/NYP/IOTA/IoT_Weather_project/model-training/datasets/official_data.csv"
df_hourly_clean, X, y = preprocess_weather_csv_hourly(csv_path)
X_train, y_train, X_val, y_val, X_test, y_test = time_split(X, y)

print("Hourly rows:", len(df_hourly_clean), "| X rows:", len(X))

# Define models (multi-output)
xgb_model = MultiOutputRegressor(
    XGBRegressor(
        n_estimators=900,
        learning_rate=0.03,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1,
    )
)

cat_model = MultiOutputRegressor(
    CatBoostRegressor(
        iterations=1200,
        learning_rate=0.03,
        depth=8,
        loss_function="RMSE",
        verbose=0,
        random_seed=42,
    )
)

lgbm_model = MultiOutputRegressor(
    LGBMRegressor(
        n_estimators=800,
        learning_rate=0.03,
        num_leaves=63,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42,
    )
)

# Train
xgb_model.fit(X_train, y_train)
cat_model.fit(X_train, y_train)
lgbm_model.fit(X_train, y_train)

print("✅ Trained XGB, CatBoost, LightGBM")



/var/folders/_r/50s4djvj3g5_yz8zn0m59nbm0000gn/T/ipykernel_59704/159949500.py:37: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



Hourly rows: 207 | X rows: 201
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000158 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 879
[LightGBM] [Info] Number of data points in the train set: 140, number of used features: 21
[LightGBM] [Info] Start training from score 35.952115
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

In [15]:

# =========================
# 4) Build ensemble (UI-friendly)
#    temp = XGB
#    pres = LGBM
#    humi = blend of XGB & CatBoost
#    choose blend weight on VAL by RMSE_mean (or humi RMSE)
# =========================
def predict_ui_ensemble(X_input, humi_w_xgb):
    px = xgb_model.predict(X_input)
    pc = cat_model.predict(X_input)
    pl = lgbm_model.predict(X_input)

    final = np.zeros_like(px)

    # temp from XGB
    final[:, 0] = px[:, 0]

    # humi blend
    w = float(humi_w_xgb)
    final[:, 1] = w * px[:, 1] + (1 - w) * pc[:, 1]

    # pres from LGBM
    final[:, 2] = pl[:, 2]

    return final



In [16]:

# Find best humidity weight using VAL
best = None
for w in np.linspace(0, 1, 21):  # 0.00, 0.05, ..., 1.00
    pred_val = predict_ui_ensemble(X_val, humi_w_xgb=w)
    m = compute_metrics(y_val, pred_val)

    # Optimize for overall RMSE_mean (smooth + avoids big misses)
    score = m["RMSE_mean"]
    if (best is None) or (score < best["score"]):
        best = {"w": w, "score": score, **m}

print("\n✅ Best humi blend weight (chosen on VAL):", best["w"])
print("VAL metrics:", {k: round(v, 4) for k, v in best.items() if k in ["MAE_mean","RMSE_mean","R2_mean","MAE_humi","RMSE_humi","R2_humi"]})


# Evaluate ensemble on TEST using best weight
pred_test_ens = predict_ui_ensemble(X_test, humi_w_xgb=best["w"])
test_metrics = compute_metrics(y_test, pred_test_ens)

print("\n===== UI-Ensemble TEST Metrics =====")
for k in ["R2_mean","MAE_mean","RMSE_mean","R2_temp","MAE_temp","RMSE_temp","R2_humi","MAE_humi","RMSE_humi","R2_pres","MAE_pres","RMSE_pres"]:
    print(f"{k:10s}: {test_metrics[k]:.4f}")



✅ Best humi blend weight (chosen on VAL): 0.9
VAL metrics: {'MAE_mean': 0.4354, 'RMSE_mean': np.float64(0.5884), 'R2_mean': 0.8215, 'MAE_humi': 0.6527, 'RMSE_humi': np.float64(0.8405), 'R2_humi': 0.8706}

===== UI-Ensemble TEST Metrics =====
R2_mean   : 0.7330
MAE_mean  : 0.5445
RMSE_mean : 0.7842
R2_temp   : 0.6267
MAE_temp  : 0.3949
RMSE_temp : 0.4822
R2_humi   : 0.7014
MAE_humi  : 0.9090
RMSE_humi : 1.1963
R2_pres   : 0.8710
MAE_pres  : 0.3296
RMSE_pres : 0.4259


In [17]:
import numpy as np
from sklearn.metrics import mean_squared_error

def walk_forward_best_weight(X_all, y_all, candidate_ws, n_folds=4):
    """
    Splits the time series into n_folds sequential validation blocks.
    For each fold, tune weight on that fold and average the score.
    """
    n = len(X_all)
    fold_size = n // (n_folds + 1)
    scores = {w: [] for w in candidate_ws}

    for k in range(1, n_folds + 1):
        # train up to k*fold_size, validate on next fold_size
        end_train = k * fold_size
        end_val = min((k + 1) * fold_size, n)

        X_tr = X_all.iloc[:end_train]
        y_tr = y_all.iloc[:end_train]
        X_va = X_all.iloc[end_train:end_val]
        y_va = y_all.iloc[end_train:end_val]

        # train models fresh per fold (more correct)
        xgb = MultiOutputRegressor(XGBRegressor(
            n_estimators=900, learning_rate=0.03, max_depth=6,
            subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0,
            objective="reg:squarederror", random_state=42, n_jobs=-1
        ))
        cat = MultiOutputRegressor(CatBoostRegressor(
            iterations=1200, learning_rate=0.03, depth=8,
            loss_function="RMSE", verbose=0, random_seed=42
        ))
        lgb = MultiOutputRegressor(LGBMRegressor(
            n_estimators=800, learning_rate=0.03, num_leaves=63,
            subsample=0.9, colsample_bytree=0.9, random_state=42
        ))

        xgb.fit(X_tr, y_tr)
        cat.fit(X_tr, y_tr)
        lgb.fit(X_tr, y_tr)

        px = xgb.predict(X_va)
        pc = cat.predict(X_va)
        pl = lgb.predict(X_va)

        for w in candidate_ws:
            pred = np.zeros_like(px)
            pred[:,0] = px[:,0]
            pred[:,1] = w*px[:,1] + (1-w)*pc[:,1]
            pred[:,2] = pl[:,2]
            rmse = np.sqrt(mean_squared_error(y_va, pred))
            scores[w].append(rmse)

    # average rmse across folds
    avg_scores = {w: float(np.mean(v)) for w, v in scores.items()}
    best_w = min(avg_scores, key=avg_scores.get)
    return best_w, avg_scores

ws = np.linspace(0, 1, 21)
best_w, avg_scores = walk_forward_best_weight(X, y, ws, n_folds=4)
print("Best w (walk-forward):", best_w)
print("Avg RMSE per w:", avg_scores)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000169 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25
[LightGBM] [Info] Number of data points in the train set: 40, number of used features: 2
[LightGBM] [Info] Start training from score 30.061989
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

### XGBoost For all 3 metrics

In [24]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

def train_xgb_3models_robust(
    X_train, y_train, X_val, y_val,
    random_state=42,
    n_estimators_grid=(200, 400, 800, 1200, 1600, 2000),
):
    params = dict(
        learning_rate=0.03,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        objective="reg:squarederror",
        random_state=random_state,
        n_jobs=-1,
        tree_method="hist",
    )

    targets = ["temp", "humi", "pres"]
    models = {}
    best_info = {}

    for i, t in enumerate(targets):
        best_rmse = None
        best_model = None
        best_n = None

        ytr = y_train.iloc[:, i]
        yva = y_val.iloc[:, i]

        for n in n_estimators_grid:
            m = XGBRegressor(n_estimators=n, **params)
            m.fit(X_train, ytr)  # no early stopping args

            pred_val = m.predict(X_val)
            rmse = np.sqrt(mean_squared_error(yva, pred_val))

            if (best_rmse is None) or (rmse < best_rmse):
                best_rmse = rmse
                best_model = m
                best_n = n

        models[t] = best_model
        best_info[t] = {"best_n_estimators": best_n, "val_rmse": float(best_rmse)}

    return models, best_info


In [25]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def predict_xgb_3models(models, X_input):
    pred_temp = models["temp"].predict(X_input)
    pred_humi = models["humi"].predict(X_input)
    pred_pres = models["pres"].predict(X_input)
    return np.column_stack([pred_temp, pred_humi, pred_pres])

def eval_preds(y_true, y_pred, name="XGB_3models"):
    out = {
        "model": name,
        "MAE_mean": mean_absolute_error(y_true, y_pred),
        "RMSE_mean": np.sqrt(mean_squared_error(y_true, y_pred)),
        "R2_mean": r2_score(y_true, y_pred, multioutput="uniform_average"),
    }
    targets = ["temp", "humi", "pres"]
    for i, t in enumerate(targets):
        out[f"MAE_{t}"] = mean_absolute_error(y_true.iloc[:, i], y_pred[:, i])
        out[f"RMSE_{t}"] = np.sqrt(mean_squared_error(y_true.iloc[:, i], y_pred[:, i]))
        out[f"R2_{t}"] = r2_score(y_true.iloc[:, i], y_pred[:, i])
    return out


In [26]:
# You already have: X_train, y_train, X_val, y_val, X_test, y_test

xgb_models, best_info = train_xgb_3models_robust(
    X_train, y_train, X_val, y_val,
    n_estimators_grid=(200, 400, 800, 1200, 1600, 2000)
)

print("Chosen n_estimators per target:")
print(best_info)

pred_val  = predict_xgb_3models(xgb_models, X_val)
pred_test = predict_xgb_3models(xgb_models, X_test)

val_metrics  = eval_preds(y_val, pred_val,  name="XGB_3models (val)")
test_metrics = eval_preds(y_test, pred_test, name="XGB_3models (test)")

import pandas as pd
display(pd.DataFrame([val_metrics, test_metrics]))


Chosen n_estimators per target:
{'temp': {'best_n_estimators': 400, 'val_rmse': 0.41996150084254097}, 'humi': {'best_n_estimators': 800, 'val_rmse': 0.8423114320170875}, 'pres': {'best_n_estimators': 800, 'val_rmse': 0.4297439585626563}}


,model,MAE_mean,RMSE_mean,R2_mean,MAE_temp,RMSE_temp,R2_temp,MAE_humi,RMSE_humi,R2_humi,MAE_pres,RMSE_pres,R2_pres
0,XGB_3models (val),0.454719,0.597366,0.814747,0.350075,0.419962,0.700337,0.661055,0.842311,0.870059,0.353023,0.429744,0.873847
1,XGB_3models (test),0.513571,0.748117,0.745418,0.395229,0.482271,0.626636,0.843155,1.127008,0.735018,0.302333,0.419891,0.874598


/Users/saikeerthan/Coding/NYP/IOTA/IoT_Weather_project/model-training/code


In [ ]:
import joblib

model_location = Path("/Users/saikeerthan/Coding/NYP/IOTA/IoT_Weather_project/model-training/artefacts/XGB")
joblib.dump(xgb_models, "xgb_3models_robust.joblib")

['xgb_3models_robust.joblib']

## ML Graph

In [30]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def _time_feats(ts: pd.Timestamp):
    hour = ts.hour
    hour_sin = np.sin(2*np.pi*hour/24.0)
    hour_cos = np.cos(2*np.pi*hour/24.0)
    dow = ts.dayofweek
    return hour_sin, hour_cos, dow


def _make_feature_row(series_df: pd.DataFrame, ts: pd.Timestamp):
    """
    series_df must have columns: temp, humi, pres
    and be hourly indexed, ending at ts - 1 hour (latest known/predicted).
    Builds ONE feature row for timestamp ts (to predict values at ts).
    """
    # We need at least 6 points for roll6_mean
    # Use last values from series_df (which includes predicted values too)
    temp = series_df["temp"]
    humi = series_df["humi"]
    pres = series_df["pres"]

    hour_sin, hour_cos, dow = _time_feats(ts)

    feat = {}

    # Current (t-1) level features not used directly in your training pipeline,
    # but your X includes raw temp/humi/pres columns (the base columns).
    # In your pipeline, X included temp/humi/pres at time t (current hour),
    # and target was t+1. For recursive forecasting, we mirror that:
    # feature "temp/humi/pres" at time ts-1 used to predict ts.
    feat["temp"] = float(temp.iloc[-1])
    feat["humi"] = float(humi.iloc[-1])
    feat["pres"] = float(pres.iloc[-1])

    feat["hour_sin"] = float(hour_sin)
    feat["hour_cos"] = float(hour_cos)
    feat["dow"] = int(dow)

    # Lags based on series_df end (which corresponds to ts-1)
    for c, s in [("temp", temp), ("humi", humi), ("pres", pres)]:
        feat[f"{c}_lag1"] = float(s.iloc[-1])
        feat[f"{c}_lag2"] = float(s.iloc[-2])
        feat[f"{c}_lag3"] = float(s.iloc[-3])
        feat[f"{c}_roll3_mean"] = float(s.iloc[-3:].mean())
        feat[f"{c}_roll6_mean"] = float(s.iloc[-6:].mean())

    return pd.DataFrame([feat], index=[ts])


def forecast_next_hours_autoreg(df_hourly_clean: pd.DataFrame, xgb_models: dict, horizon_hours: int = 24):
    """
    Autoregressive forecasting:
    - Start from the last available historical hour
    - Predict 1 hour ahead, append prediction
    - Repeat for horizon_hours

    Returns:
    - hist_df: historical df (copy)
    - fcst_df: forecast df with columns temp/humi/pres, indexed by future timestamps
    """
    hist_df = df_hourly_clean[["temp", "humi", "pres"]].copy()

    # Need enough history to compute lag3 and roll6
    if len(hist_df) < 6:
        raise ValueError("Need at least 6 hourly points in df_hourly_clean for roll6 features.")

    # This "series_df" will grow with predictions
    series_df = hist_df.copy()

    last_ts = series_df.index.max()
    future_index = pd.date_range(start=last_ts + pd.Timedelta(hours=1), periods=horizon_hours, freq="1H")

    preds = []

    for ts in future_index:
        X_row = _make_feature_row(series_df, ts)

        # Predict each target using its model
        temp_pred = float(xgb_models["temp"].predict(X_row)[0])
        humi_pred = float(xgb_models["humi"].predict(X_row)[0])
        pres_pred = float(xgb_models["pres"].predict(X_row)[0])

        preds.append((temp_pred, humi_pred, pres_pred))

        # Append predicted row so next step can use it
        series_df.loc[ts, ["temp", "humi", "pres"]] = [temp_pred, humi_pred, pres_pred]

    fcst_df = pd.DataFrame(preds, index=future_index, columns=["temp", "humi", "pres"])
    return hist_df, fcst_df


def plot_history_with_continuation(hist_df: pd.DataFrame, fcst_df: pd.DataFrame, title="Weather Forecast (Hourly)"):
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.08,
        subplot_titles=("Temperature (°C)", "Humidity (%)", "Pressure (hPa)")
    )

    series = [("temp", 1), ("humi", 2), ("pres", 3)]
    for col, r in series:
        # Historical
        fig.add_trace(
            go.Scatter(
                x=hist_df.index, y=hist_df[col],
                mode="lines",
                name=f"{col} (history)",
            ),
            row=r, col=1
        )

        # Forecast continuation
        fig.add_trace(
            go.Scatter(
                x=fcst_df.index, y=fcst_df[col],
                mode="lines",
                name=f"{col} (forecast)",
                line=dict(dash="dash")  # visually distinct continuation
            ),
            row=r, col=1
        )

        # Optional: a vertical line at the boundary (last historical timestamp)
        boundary = hist_df.index.max()
        fig.add_vline(x=boundary, line_width=1, line_dash="dot", row=r, col=1)

    fig.update_layout(
        title=title,
        height=900,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        margin=dict(l=40, r=40, t=80, b=40),
    )
    fig.update_xaxes(title_text="Time", row=3, col=1)
    return fig


# ======== USAGE ========
# Requires: df_hourly_clean and xgb_models already defined
hist_df, fcst_df = forecast_next_hours_autoreg(df_hourly_clean, xgb_models, horizon_hours=24)

fig = plot_history_with_continuation(hist_df, fcst_df, title="Hourly Weather: History + 24h Forecast Continuation")
fig.show()


/var/folders/_r/50s4djvj3g5_yz8zn0m59nbm0000gn/T/ipykernel_59704/3281698521.py:76: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

